# Import Libraries

In [9]:
!pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [4]:
import os
import pandas as pd
import torch
import numpy as np
from Preprocess import Process
from NeuralNetworkDIY import ClassifierNetwork, Model
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image



In [5]:
print(torch.cuda.is_available())

True


# Parameters for the dataset

In [6]:
RAW_DIR = "data/raw"
PROCESSED_DIR = "data/processed"
IMAGE_SIZE=(128,128)

# Image Preprocessing

In [7]:
process=Process(RAW_DIR, PROCESSED_DIR, IMAGE_SIZE, delete_process_dir=True, batch_size=1000, aug_amount=0, filters=False)
process.run(0.7, 0.3, 1)

Working on fake


Processing train:   0%|          | 0/112 [00:07<?, ?it/s]

KeyboardInterrupt



# DataLoader and Custom Dataset

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.image_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        transform = transforms.ToTensor()
        
        img= transform(img)
        # Extract label from the filename (fake -> 1, real -> 0)
        label = 1 if "fake" in os.path.basename(img_path).lower() else 0

        return img, label, img_path

In [ ]:
train_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/train/rgb')
valid_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/valid/rgb')

train_loader = DataLoader(train_data, batch_size=1000, shuffle=True) #batchsize set to 1000, GPU can handle it, maybe more too
valid_loader = DataLoader(valid_data, batch_size=1000, shuffle=False)

# Creating the network, model, training, predicting

In [ ]:
network=ClassifierNetwork(IMAGE_SIZE)

In [ ]:
model=Model(network, 20)

In [ ]:
model.train(train_loader)

In [ ]:
results=model.predict(valid_loader)

In [35]:
def confusion_matrix(results):
    df = pd.DataFrame(results).transpose()
    length=len(df)
    df_agg = df.groupby(0)[1].agg(
        _0=lambda x: (x == 0).sum(),
        _1=lambda x: (x == 1).sum()
    ).reset_index()

    true_positives = df_agg[df_agg[0] == 1]['_1'].sum()  # True Positive
    true_negatives = df_agg[df_agg[0] == 0]['_0'].sum()  # True Negative
    false_positives = df_agg[df_agg[0] == 0]['_1'].sum()  # False Positive
    false_negatives = df_agg[df_agg[0] == 1]['_0'].sum()  # False Negative

    matrix = pd.DataFrame({
        'True 0': [true_negatives, false_positives],
        'True 1': [false_negatives, true_positives]
    }, index=['Pred 0', 'Pred 1'])
    
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    
    return matrix, accuracy


In [36]:
matrix, accuracy = confusion_matrix(results)

In [39]:
matrix

,True 0,True 1
Pred 0,18076,511
Pred 1,2924,47489


In [40]:
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9502
